In [ ]:
!pip install transformers

In [ ]:
!pip install simpletransformers

In [ ]:
#predicting
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [ ]:
final_news_df = pd.read_csv('/content/drive/MyDrive/final_news_df_fin.csv')

In [ ]:
final_news_df.clean_headline = final_news_df.clean_headline.fillna('')

In [ ]:
# Load the saved model
model_dir = 'yiyanghkust/finbert-tone'
model = TFBertForSequenceClassification.from_pretrained(model_dir, num_labels=3,ignore_mismatched_sizes=True)

df = final_news_df

# Tokenize the text using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = []
for _, row in df.iterrows():
    tokenized_text = tokenizer.tokenize(row[df.columns[4]]) # change the "df.columns[1]" to whatver your column number is.
    tokenized_texts.append(tokenized_text)

# Convert text to input features compatible with BERT
input_ids = []
attention_masks = []
for tokens in tokenized_texts:
    inputs = tokenizer.encode_plus(
        ' '.join(tokens),  # Join the tokens into a string
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Make predictions using the saved model
input_ids = input_ids.numpy()
attention_masks = attention_masks.numpy()
outputs = model.predict([input_ids, attention_masks])
predictions = np.argmax(outputs[0], axis=1)
# scores = np.argmax(outputs[1], axis=1)

# Add the predictions to the dataframe
final_news_df['predictions']= predictions # the predicted sentiments will be stored in a new column named "my_sentiment"
# final_news_df['pred_scores']= scores
# Save the dataframe as a CSV file
# output_file = "InSamp_final_finbert.csv"
# final_news_df.to_csv(output_file, index=False)

Some layers from the model checkpoint at yiyanghkust/finbert-tone were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


358/358 [==============================] - 215s 571ms/step


In [ ]:
outputs[0]

TFSequenceClassifierOutput(loss=None, logits=array([[-5.693507 , 10.599956 , -5.6275463],
       [-2.7757354,  6.4353065, -4.3515368],
       [-4.817431 ,  9.611035 , -5.2125845],
       ...,
       [-5.5682235, 10.836731 , -5.857997 ],
       [-4.451083 ,  9.318366 , -5.2299595],
       [-5.6892686, 10.695096 , -5.294671 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
import scipy

In [ ]:
logits = scipy.special.softmax(np.array(outputs[0]))
scores = pd.Series(logits[:, 0] - logits[:, 1])

In [ ]:
final_news_df['pred_scores']= scores
# output_file = "InSamp_final_finbert.csv"
final_news_df.to_csv("InSamp_final_finbert.csv")

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
final_news_df.sent_score = final_news_df.sent_score.fillna(0)

In [ ]:
print("accuracy score: ", accuracy_score(final_news_df['sent_score'],predictions))

accuracy score:  0.30330304089479204


In [ ]:
final_news_df_out = pd.read_csv('/content/drive/MyDrive/final_news_df_out_fin.csv')

In [ ]:
final_news_df_out.columns

Index(['Unnamed: 0', 'week', 'FROM', 'TO', 'headline', 'clean_headline',
       'returns', 'tf-idf', 'sentiment', 'sent_score', 'compname'],
      dtype='object')

In [ ]:
final_news_df_out.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# #predicting
# import pandas as pd
# import numpy as np
# from transformers import BertTokenizer, TFBertForSequenceClassification
# import tensorflow as tf
# import warnings
# warnings.filterwarnings("ignore")


final_news_df_out = pd.read_csv('/content/drive/MyDrive/final_news_df_out_fin.csv')
final_news_df_out.drop('Unnamed: 0', axis=1, inplace=True)
final_news_df_out.clean_headline = final_news_df_out.clean_headline.fillna('')
df_out = final_news_df_out

# Tokenize the text using BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = []
for _, row in df_out.iterrows():
    # print(row[df_out.columns[4]])
    tokenized_text = tokenizer.tokenize(row[df_out.columns[4]]) # change the "df.columns[1]" to whatver your column number is.
    tokenized_texts.append(tokenized_text)

# Convert text to input features compatible with BERT
input_ids = []
attention_masks = []
for tokens in tokenized_texts:
    inputs = tokenizer.encode_plus(
        ' '.join(tokens),  # Join the tokens into a string
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Make predictions using the saved model
input_ids = input_ids.numpy()
attention_masks = attention_masks.numpy()
outs = model.predict([input_ids, attention_masks])
predictions_out = np.argmax(outs[0], axis=1)
logits_out = scipy.special.softmax(np.array(outs[0]))
scores_out = pd.Series(logits_out[:, 0] - logits_out[:, 1])
# Add the predictions to the dataframe
final_news_df_out['predictions']= predictions_out
final_news_df_out['pred_scores']= scores_out

# output_file = "OutSamp_final_finbert.csv"
final_news_df.to_csv("OutSamp_final_finbert.csv", index=False)

print("accuracy score: ", accuracy_score(final_news_df_out['sent_score'],predictions_out))

227/227 [==============================] - 130s 574ms/step
accuracy score:  0.3560606060606061
